In [ ]:
import numpy as np
import numpy.ma as ma  # masked array
import scipy.stats as sta # statistics
from scipy.special import (erf, erfinv)

from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.models import ColumnDataSource, Whisker, Label, LabelSet, MultiLine
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot, column, row

from bokeh import palettes

# 2020-08-22 Para rodar curve_fit
from gauss_funcs import func_example, func_example_fixed_b
# 2020-09-03 Desativei por enquanto
# import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# 2020-08-30
from scipy.interpolate import splrep, splev

from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

# 2020-09-11
from opengamma_searchfuncs import selectOverlappedPks

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [ ]:
# hei = 0.01
# haPercWin = 9 # 2020-12-05 Meia janela para o cálculo do percentil para a linha base:
# 9 para 4k canais, 18 para 8k canais.
# yrange = [10**1, 10**5]
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'
# yrange = [10**1, 10**5]
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

# Para espectros CTP 4k
kSigma = 4.0 # Com 3.0 ele pega os grandes degraus Compton, com 4.0 só o da esquerda (retroespalhamento)
aSta = 0.004
bSta = 3.0
kExpand = 1 # Parâmetro para alargar região (inteiro positivo)
# 2020-12-05 Ainda não aprendi a mexer com o smoo... Chutando bem baixo para 4k, altíssima intens
smoo = 0.1 # Tem que ser >= 0.0
yrange = [10**1, 10**6]
fn2 = '../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

# Para espectros Almera 8k
# kSigma = 3.0 # Maior, menos sensível
# aSta = 0.003 # Janela para estatísticas
# bSta = 10.0
# kExpand = 1 # Parâmetro para alargar região (inteiro positivo)
# smoo = 15
# yrange = [10**1, 10**5]

# Alguns espectros ALMERA com relatórios do Genie2k em .REP:
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp3(QC)_F100_16_Cont-A.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp2_spk_water_16_250k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp4_f100_8k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp4_f100_63k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp5_F100_16_Cont-A.IEC'

# kSigma = 2.0
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

# yrange = [10**1, 10**6]
# fn2 = './I125_01.001.3.IEC'

# 2020-12-05 Espectros do tq de retenção do CEN: 54 e 55 de 2020.
# yrange = [10**1, 10**3]
# fn2 = '../Genie_Transfer/Si/SI2020/SI05420.Chn'

# yrange = [10**1, 10**5]
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'

# sigma 5 funciona para este
# yrange = [10**1, 10**6]
# fn2 = './I125_01.001.3.IEC'

# 13 de julho de 2020
# PAREI AQUI O termo de 1o grau oscila selvagemente em baixas contagens
# ponderá-lo pelo coeffs0
# Para o espectro do I-123, usar kSigma = 5.0